In [5]:
%%capture output
!pip install ipywidgets
!pip install jupyterlab_widgets
# !pip install nodejs
# !pip install npm
!pip install widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

In [6]:
import ipywidgets as widgets

In [12]:
w = widgets.IntSlider()

In [13]:
display(w)

IntSlider(value=0)

In [14]:
display(w)

IntSlider(value=0)

In [15]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

In [17]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', options=('1', '2', '3'), value='1')